In [1]:
import pandas as pd

BOOKS_FILE = 'books_polarity_rating.jsonlines'

books = pd.read_json(BOOKS_FILE, lines=True, nrows=10000)

In [2]:
books.head()

,book_id,type,title,titleEnglish,description,attrs,rating,categories,NEGATIVE,POSITIVE,polarity_rating
0,0,book,"Їсти, молитися, кохати","Eat, Pray, Love: One Woman's Search for Everyt...","«Їсти, молитися, кохати» - надзвичайно щира ро...","{'Автор': 'Елізабет Ґілберт', 'Видавництво': '...",4.6,"[Книжки, Художня література, Романтична проза,...",6.0,24.0,4.0
1,1,book,Серія книжок про Джуді Муді (комплект із 10 книг),None,На вас чекає знайомство з кумедною дівчинкою н...,"{'Автор': 'Меґан МакДоналд', 'Видавництво': 'В...",5.0,"[Книжки, Дитяча література, Пригоди]",NaN,NaN,NaN
2,2,book,Три новеллы (комплект из 2 книг),Och Varje Morgon Blir Vagen. Hem Langre och La...,Новели Фредріка Бакмана пройняті тим же м'яким...,"{'Автор': 'Фредрік Бакман', 'Видавництво': 'Си...",5.0,"[Книжки, Художня література, Сучасна проза]",NaN,NaN,NaN
3,3,book,Тренажер мозга. Как развить гибкость мышления ...,"Brain Coach. Train, Regain and Maintain Your M...",Ви - це ваш мозок. Ваш мозок має неймовірний п...,"{'Автор': 'Гарет Мур', 'Видавництво': 'Альпіна...",NaN,"[Книжки, Саморозвиток. Мотивація, Покращення п...",NaN,NaN,NaN
4,4,book,У пошуках Івана Сили,None,"Документальні нариси Олександра Гавроша, автор...","{'Автор': 'Олександр Гаврош', 'Видавництво': '...",NaN,"[Книжки, Біографії й мемуари, Спортсмени]",NaN,NaN,NaN


In [3]:
import re

def senitize_description(description):
    return re.sub("[^\w ]", "", description)

sanitized_description_col = books['description'].apply(senitize_description)
books = books.assign(sanitized_description_col = sanitized_description_col)

In [4]:
# Utilities

def get_recomendations(recomendations_df, books, book_id):
    similarity_indexes = recomendations_df[book_id].sort_values(ascending=False).round(3)
    book_ids_recomended = similarity_indexes.index[:10]

    return books[(books['book_id'].isin(book_ids_recomended))] \
        .reindex(book_ids_recomended)[['book_id', 'title', 'titleEnglish', 'polarity_rating']] \
        .assign(similarity_index = similarity_indexes) \
        .sort_values(['similarity_index', 'polarity_rating'], ascending=[False, False])

def get_book_title_by_id(books, book_id):
    return books[(books['book_id'] == book_id)]['title'].iloc[0]

CORPUS = books['sanitized_description_col']
BOOKS = books
BOOK_ID = 12

In [5]:
# Recomendation based on the count words

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def create_count_word_recomendations(corpus):
    cv = CountVectorizer()
    word_count_matrix = cv.fit_transform(corpus)
    word_count_documents_similarity = cosine_similarity(word_count_matrix, dense_output = True)
    word_count_recomendations_df = pd.DataFrame(word_count_documents_similarity)
    return word_count_recomendations_df

def count_words_recomendation_demo():
    book_title = get_book_title_by_id(BOOKS, BOOK_ID)
    print(f"Book title: '{book_title}'")

    count_word_recomendations_df = create_count_word_recomendations(CORPUS)
    recomendations = get_recomendations(count_word_recomendations_df, BOOKS, BOOK_ID)
    display(recomendations)

count_words_recomendation_demo()

Book title: 'Большая книга пекаря. Хлеб, бриоши, выпечка. Учимся готовить шедевры'


,book_id,title,titleEnglish,polarity_rating,similarity_index
12,12,"Большая книга пекаря. Хлеб, бриоши, выпечка. У...",Le Grand Manuel du Boulanger,5.0,1.000
9647,9647,Телекомунікаційні системи та мережі. Принципи ...,None,NaN,0.236
4543,4543,Матеріалознавство для архітекторів та дизайнерів,None,NaN,0.223
416,416,Кишенькова книжка емоційного інтелекту. Невели...,Emotional Intelligence Pocketbook: Little Exer...,5.0,0.213
4244,4244,Бетмен. Цить,Batman. Hush,0.0,0.200
6470,6470,Інтернет речей,None,NaN,0.194
6007,6007,Чарiвна Україна / Magic Ukraine,None,NaN,0.191
6654,6654,Сервер на основі операційної системи FreeBSD 6.1,None,NaN,0.190
6460,6460,Дивовижні технології. Дизайн та інтернет речей,"Enchanted Objects: Design, Human Desire, and t...",NaN,0.186
6899,6899,В Землю #1,None,NaN,0.186


In [6]:
# Recomendation based on the tfidf

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def create_tfidf_recomendations(corpus):
    tfidf = TfidfVectorizer(ngram_range=(1, 3))
    tfidf_matrix = tfidf.fit_transform(corpus)
    tfidf_similarity_matrix = cosine_similarity(tfidf_matrix)
    tfidf_recomendation_df = pd.DataFrame(tfidf_similarity_matrix)
    return tfidf_recomendation_df

def tfidf_recomendation_demo():
    book_title = get_book_title_by_id(BOOKS, BOOK_ID)
    print(f"Book title: '{book_title}'")
    
    tfidf_recomendations_df = create_tfidf_recomendations(CORPUS)
    recomendations = get_recomendations(tfidf_recomendations_df, BOOKS, BOOK_ID)
    display(recomendations)

tfidf_recomendation_demo()

Book title: 'Большая книга пекаря. Хлеб, бриоши, выпечка. Учимся готовить шедевры'


,book_id,title,titleEnglish,polarity_rating,similarity_index
12,12,"Большая книга пекаря. Хлеб, бриоши, выпечка. У...",Le Grand Manuel du Boulanger,5.0,1.000
199,199,Випічка. Десерти,None,5.0,0.024
2634,2634,Выразительный человек. Сценическое воспитание ...,None,NaN,0.016
416,416,Кишенькова книжка емоційного інтелекту. Невели...,Emotional Intelligence Pocketbook: Little Exer...,5.0,0.015
4981,4981,Классика ХХI века. Фортепианная игра. Ответы н...,None,NaN,0.015
329,329,Игры тестостерона и другие вопросы биологии по...,Trouble With Testosterone. And Other Essays on...,NaN,0.015
224,224,Солодка неділя,None,5.0,0.014
209,209,Готовим в стиле хюгге,None,NaN,0.014
9647,9647,Телекомунікаційні системи та мережі. Принципи ...,None,NaN,0.014
658,658,Англійська з Бреді. Level 2 (комплект з 5 книг),None,NaN,0.013
